# Modellutveckling och träning 
Data förbereds för att kunna använda inför träning av olika regressionsmodeller som ska förutsäga taxipris.

### Träning av linjär regressionsmodell 

Vi tränar vår första modell som jämförelsepunkt. 
Den ger oss en baslinje för hur bra det enklaste modellerna kan prestera 

In [28]:
import pandas as pd 
from taxipred.utils.constants import DATA_PATH

df = pd.read_csv(DATA_PATH / "taxi_trip_clean.csv")
df.head(10)

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,53.82,36.2624
1,36.87,Evening,Weekend,1.0,High,Clear,37.27,52.9032
2,8.64,Afternoon,Weekend,2.0,Medium,Clear,89.33,60.2028
3,30.45,Morning,Weekday,3.0,High,Clear,110.33,NaN
4,41.79,Night,Weekend,3.0,High,Clear,86.95,88.1328
5,9.91,Evening,Weekday,2.0,High,Clear,41.72,28.9914
6,9.99,Night,Weekday,4.0,High,Clear,34.00,27.4415
7,15.91,Morning,Weekday,4.0,Low,Clear,114.93,56.7160
8,26.71,Afternoon,Weekend,4.0,Low,Rain,111.18,69.0049
9,30.98,Afternoon,Weekend,1.0,Low,Rain,57.02,44.7782


Vi tar bort rader där Trip_Price saknas eftersom modellen behöver ett faktiskt pris för att kunna tränas.

In [29]:
df_clean = df[df["Trip_Price"].notna()]
df_clean.head(5)

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,53.82,36.2624
1,36.87,Evening,Weekend,1.0,High,Clear,37.27,52.9032
2,8.64,Afternoon,Weekend,2.0,Medium,Clear,89.33,60.2028
4,41.79,Night,Weekend,3.0,High,Clear,86.95,88.1328
5,9.91,Evening,Weekday,2.0,High,Clear,41.72,28.9914


Här plockar vi ut de rader som saknar Trip_Price.
Det används inte i träningen, men vi sparar dem för att kunna testa den färdiga modellen senare.

In [30]:
df_nan_values = df[df["Trip_Price"].isna()]
df_nan_values.head(5)

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
3,30.45,Morning,Weekday,3.0,High,Clear,110.33,NaN
14,4.19,Morning,Weekday,1.0,Low,Clear,69.06,NaN
51,38.78,Morning,Weekday,3.0,Medium,Clear,90.14,NaN
90,10.14,Afternoon,Weekday,4.0,Low,Rain,25.41,NaN
95,46.90,Evening,Weekday,1.0,High,Clear,66.38,NaN


## Linjär regression
I detta steg väljer vi vilket värde modellen ska förutsäga Trip_Price och vilka features den ska använda som input.

In [31]:
X,y = df_clean.drop ("Trip_Price", axis="columns"), df_clean["Trip_Price"]

X.head(), y.head()

(   Trip_Distance_km Time_of_Day Day_of_Week  Passenger_Count  \
 0             19.35     Morning     Weekday              3.0   
 1             36.87     Evening     Weekend              1.0   
 2              8.64   Afternoon     Weekend              2.0   
 4             41.79       Night     Weekend              3.0   
 5              9.91     Evening     Weekday              2.0   
 
   Traffic_Conditions Weather  Trip_Duration_Minutes  
 0                Low   Clear                  53.82  
 1               High   Clear                  37.27  
 2             Medium   Clear                  89.33  
 4               High   Clear                  86.95  
 5               High   Clear                  41.72  ,
 0    36.2624
 1    52.9032
 2    60.2028
 4    88.1328
 5    28.9914
 Name: Trip_Price, dtype: float64)

Här skapar vi X-datan för de rader som saknar Trip_Price.
Dessa kommer senare användas för att testa vår slutmodell på verkliga fall där priset saknas.

In [32]:
X_nan_values = df_nan_values.drop(columns="Trip_Price")
X_nan_values.head(5)

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes
3,30.45,Morning,Weekday,3.0,High,Clear,110.33
14,4.19,Morning,Weekday,1.0,Low,Clear,69.06
51,38.78,Morning,Weekday,3.0,Medium,Clear,90.14
90,10.14,Afternoon,Weekday,4.0,Low,Rain,25.41
95,46.90,Evening,Weekday,1.0,High,Clear,66.38


### Train/Test-split 
Här delas datat upp i träningsdata och testdata för att kunna utvärdera modellernas prestanda på ny data

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)

# help(train_test_split)

X_train.shape, X_test.shape 

((367, 7), (181, 7))

För att kunna bygga en pipeline behöver vi dela upp våra features i numeriska och kategoriska kolumner.
Det gör att vi kan använda rätt typ av förbehandling på varje datatyp (skalning för numeriska och One-Hot Encoding för kategoriska)

In [34]:
int_cols = X_train.select_dtypes(include="number").columns
object_cols = X_train.select_dtypes(exclude="number").columns

Här skapar vi en ColumnTransformer som automatiskt för behandlar olika datatyper

- Numeriska kolumner skalas med StandardScaler
- Kategoriska kolumner omvandlas till dummy-variabler med OneHotEncoder

Denna transformer kommer att köras innan själva modellen tränas.

In [35]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("int", StandardScaler(), int_cols),
        ("object", OneHotEncoder(handle_unknown="ignore"), object_cols),
    ]
)


Vi skapar en pipline som kombinerar två steg: 
- 1 Förbehandling av datan (preprocessor)
- 2 Träning av modellen med Linear Regression

Genom att använda en pipeline säkerställer vi att samma förbehandling används både vid träning och vid prediktion.

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

lr_model = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", LinearRegression())
])

Nu tränar vi vår linjära regressionsmodell på träningsdatan.
Pipelinen ser till att datan först förbehandlas innan modellen anpassas.

In [37]:
lr_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('int', ...), ('object', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transfor

### Förutsägelse med modellen 

Här använder vi vår tränade modell för att förutsäga taxipris på testdatan.
Vi visar även de fem första prediktionerna för att snabbt få en överblick av resultatet.

In [38]:
y_pred_lr = lr_model.predict(X_test)
y_pred_lr[:5]

array([74.81221864, 37.36059963, 84.955874  , 24.09907271, 42.12206597])

### Utvärdering av modellen 

In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np 

mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr  = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)

print(f"MAE: {mae_lr:.2f}, MSE: {mse_lr:.2f}, RMSE: {rmse_lr:.2f}")


MAE: 11.16, MSE: 210.52, RMSE: 14.51


### Utvärdering av modellen

Modellens prestanda utvärderas med tre vanliga felmått:

- MAE (Mean Absolute Error) – genomsnittligt fel i samma enhet som priset
- MSE (Mean Squared Error) – förstärker större fel
- RMSE (Root Mean Squared Error) – mer lättolkad än MSE

Dessa värden använder vi sedan för att jämföra våra olika modeller.

# KNN (K-Nearest Neighbors)

I detta steg tränar vi en KNN-modell för att förutsäga taxipriset
KNN fungerar genom att hitta de närmaste grannarna i träningsdatan och använda deras genomsnitt för att förutsäga ett pris

KNeighborsRegressor(n_neighbors=5) skapar en modell som tittar på det 5 närmaste grannarna vid varje prediktion 

In [40]:
from sklearn.neighbors import KNeighborsRegressor

knn_model = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", KNeighborsRegressor(n_neighbors=5))
])

Modellen tränas på träningsdatan, och skalningen görs automatiskt av pipelinen.

In [41]:
knn_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('int', ...), ('object', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transfor

In [42]:
y_pred_knn = knn_model.predict(X_test)

In [43]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np 

mae_knn = mean_absolute_error(y_test, y_pred_knn)
mse_knn  = mean_squared_error(y_test, y_pred_knn)
rmse_knn = np.sqrt(mse_knn)

print(f"MAE: {mae_knn:.2f}, MSE: {mse_knn:.2f}, RMSE: {rmse_knn:.2f}")


MAE: 12.18, MSE: 237.53, RMSE: 15.41


### Utvärdering av modellen

Modellens prestanda utvärderas med tre vanliga felmått:

- MAE (Mean Absolute Error) – genomsnittligt fel i samma enhet som priset
- MSE (Mean Squared Error) – förstärker större fel
- RMSE (Root Mean Squared Error) – mer lättolkad än MSE

Dessa värden använder vi sedan för att jämföra våra olika modeller.

# Random Forest

Här skapar vi en Random forest modell med 200 träd
Random Forest fungerar genom att kombinera många besluts-träd för att få mer stabila och robusta prediktioner

## Träning av modellen 

Här skapar vi en Random Forest-modell med 200 träd.
Random Forest fungerar genom att kombinera många beslutsträd för att få mer stabila och robusta prediktioner.

In [44]:
from sklearn.ensemble import RandomForestRegressor

rf_model = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", RandomForestRegressor(n_estimators=200, random_state=42))
])

Modellen tränas på träningsdatan, och skalningen görs automatiskt av pipelinen.

In [45]:
rf_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('int', ...), ('object', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transfor

Modellen används för att förutsäga taxipriserna på testdatan. Vi visar de fem första resultaten för översikt

In [46]:
y_pred_rf = rf_model.predict(X_test)
y_pred_rf[:5]

array([74.5525715, 39.453454 , 78.855564 , 20.678126 , 43.0061125])

Här beräknar vi modellens felvärden (MAE, MSE och RMSE) för att se hur bra Random Forest presterar på testdatan.

In [47]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np 

mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"MAE: {mae_rf:.2f}, MSE: {mse_rf:.2f}, RMSE: {rmse_rf:.2f}")

MAE: 11.88, MSE: 236.11, RMSE: 15.37


In [48]:
results = pd.DataFrame({
    "Model": ["Linear Regression", "KNN", "Random Forest"],
    "MAE": [mae_lr, mae_knn, mae_rf],
    "MSE": [mse_lr, mse_knn, mse_rf], 
    "RMSE": [rmse_lr, rmse_knn, rmse_rf]
})

results

,Model,MAE,MSE,RMSE
0,Linear Regression,11.163529,210.515543,14.509154
1,KNN,12.182929,237.530461,15.412023
2,Random Forest,11.884068,236.112544,15.365954


# Val av slutmodell 
Vi har testat tre olika modeller men vi kommer att gå vidare med Linear Regression som slutmodell. 
För att den gav lägst felvärden på testdatan. 

- MAE: 11.16
- MSE: 210.52
- RMSE: 14.51


Här använder vi vår slutmodell (Linear Regression) för att göra prisförutsägelser på testdatan.
Vi visar även de första fem prediktionerna för att få en snabb överblick.

In [49]:
y_pred_final = lr_model.predict(X_test)
y_pred_final[:5]

array([74.81221864, 37.36059963, 84.955874  , 24.09907271, 42.12206597])

Här använder vi modellen för att fylla i de rader där Trip_Price saknas.
Modellen förutsäger priset baserat på övriga features.

In [50]:
price_prediction = lr_model.predict(X_nan_values)
price_prediction

array([76.622616  , 29.334512  , 79.42050239, 25.39690588, 79.06216768,
       61.29337396, 66.87925932, 69.45932458, 92.18414395, 65.14477267,
       68.52897758, 42.22487058, 25.90368599, 27.6680514 , 52.01856322,
       22.57280449, 45.64412348, 65.91844884, 27.57731021, 42.93408354,
       45.35247292, 33.49809687, 44.48353119, 44.71785634, 76.02414344,
       48.20695171])

Vi skapar en ny kolumn Predicted_Trip_Price där vi lägger in modellens förutsägelser för de rader som saknade pris.
Detta gör det möjligt att analysera och använda dessa rader senare.

In [51]:
df_nan_values = df_nan_values.copy()
df_nan_values["Predicted_Trip_Price"] = price_prediction

df_nan_values.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price,Predicted_Trip_Price
3,30.45,Morning,Weekday,3.0,High,Clear,110.33,NaN,76.622616
14,4.19,Morning,Weekday,1.0,Low,Clear,69.06,NaN,29.334512
51,38.78,Morning,Weekday,3.0,Medium,Clear,90.14,NaN,79.420502
90,10.14,Afternoon,Weekday,4.0,Low,Rain,25.41,NaN,25.396906
95,46.90,Evening,Weekday,1.0,High,Clear,66.38,NaN,79.062168


Vi slår nu ihop de rader som hade saknat pris (men som nu fått en förutsagd Trip_Price) med den ursprungliga datan.  
Därefter tar vi bort kolumnen med den predikterade versionen och sorterar datan för att få ett komplett och städat dataset.

In [52]:
df_nan_filled = df_nan_values
df_nan_filled["Trip_Price"] = df_nan_filled["Predicted_Trip_Price"]
df_nan_filled = df_nan_filled.drop(columns=["Predicted_Trip_Price"])

df_full = pd.concat([df_clean, df_nan_filled], axis=0).sort_index()
df_full.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,53.82,36.262400
1,36.87,Evening,Weekend,1.0,High,Clear,37.27,52.903200
2,8.64,Afternoon,Weekend,2.0,Medium,Clear,89.33,60.202800
3,30.45,Morning,Weekday,3.0,High,Clear,110.33,76.622616
4,41.79,Night,Weekend,3.0,High,Clear,86.95,88.132800


Vi kontrollerar nu att inga saknade värden finns kvar i vårt slutliga dataset.  
Eftersom modellen har fyllt i alla tidigare saknade Trip_Price-värden ska resultatet visa enbart nollor.

In [53]:
df_full.isna().sum()

Trip_Distance_km         0
Time_of_Day              0
Day_of_Week              0
Passenger_Count          0
Traffic_Conditions       0
Weather                  0
Trip_Duration_Minutes    0
Trip_Price               0
dtype: int64

In [54]:
df_full.to_csv(DATA_PATH/"final_data.csv", index= False)

Till sist tränar vi modellen en sista gång på hela datan och sparar den som en joblib-fil.  
Detta gör att modellen kan laddas in och användas senare utan att behöva tränas om.

In [55]:
import joblib
from taxipred.utils.constants import DATA_PATH

lr_model.fit(X, y)
joblib.dump(lr_model, DATA_PATH / "final_model.joblib", compress=("xz", 3), protocol=5)

['/Users/hazajijan/MLOPS/maskininlärningsmodeller och algoritmer/taxi-prediction-fullstack-haza/src/taxipred/data/final_model.joblib']